In [206]:
import numpy as np
import torch

from torchsummary import summary
from torchvision import models

#import albumentations as alb
# from albumentations.pytorch import ToTensor
import numpy as np

from torch import nn
from torch.nn import functional as F
from PIL import Image
from torchvision import transforms, datasets
from tqdm import tqdm

from scipy import misc
from PIL import Image
from torchvision import transforms
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split

In [207]:
train = datasets.CIFAR10(root='data/', train=True, download=True)


class MyOwnCifar(torch.utils.data.Dataset):
   
    def __init__(self, init_dataset, transform=None):
        self._base_dataset = init_dataset
        self.transform = transform

    def __len__(self):
        return len(self._base_dataset)

    def __getitem__(self, idx):
        img = self._base_dataset[idx][0]
        if self.transform is not None:
            img = self.transform(img)
        return img, self._base_dataset[idx][1]

Files already downloaded and verified


In [220]:
trans_actions_train = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomCrop(32, padding = 4),
    transforms.ToTensor(),
    transforms.transforms.Normalize(mean=[0.5, 0.5, 0.5],
                              std=[0.5, 0.5, 0.5])
])

trans_actions_valid = transforms.Compose(
    [transforms.ToTensor(),
     transforms.transforms.Normalize(mean=[0.5, 0.5, 0.5],
                              std=[0.5, 0.5, 0.5])
      ])

In [221]:
def train_valid_split(Xt):
    X_train, X_test = train_test_split(Xt, test_size=0.05, random_state=13)
    return X_train, X_test

In [222]:
def loader(data):

    train_dataset, valid_dataset = train_valid_split(train)

    train_dataset = MyOwnCifar(train_dataset, trans_actions_train)
    valid_dataset = MyOwnCifar(valid_dataset, trans_actions_valid)

    train_loader = torch.utils.data.DataLoader(train_dataset,
                              batch_size=30,
                              shuffle=False)
    valid_loader = torch.utils.data.DataLoader(valid_dataset,
                              batch_size=30,
                              shuffle=False)
    
    return train_loader, valid_loader

In [223]:
train_loader, valid_loader = loader(train)

In [224]:
class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        self.dp_one = nn.Dropout(0.2)
        self.dp_two = nn.Dropout(0.2)
        
        self.bn_one = torch.nn.BatchNorm2d(3) 
        self.conv_one = torch.nn.Conv2d(3, 30, 3)
        self.bn_two = torch.nn.BatchNorm2d(30) 
        self.conv_two = torch.nn.Conv2d(30, 60, 3)
        self.bn_three = torch.nn.BatchNorm2d(60)
        self.conv_three = torch.nn.Conv2d(60, 120, 3)
        self.bn_four = torch.nn.BatchNorm2d(120)
        self.fc1 = torch.nn.Linear(480, 200)
        self.fc2 = torch.nn.Linear(200, 60)
        self.out = torch.nn.Linear(60, 10)
        
    def forward(self, x):
        x = self.bn_one(x)
        x = self.conv_one(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2)
        
        x = self.bn_two(x)
        x = self.conv_two(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2)
        
        x = self.bn_three(x)
        x = self.conv_three(x)
        x = F.leaky_relu(x, 0.1)
        x = F.max_pool2d(x, 2)
        
        x = self.bn_four(x)
        x = x.view(x.size(0), -1)
        x = self.dp_one(x)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.dp_two(x)
        x = self.fc2(x)
        x = F.relu(x)
        return self.out(x)

In [225]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

print(device)

cpu


In [226]:
net = Net()

net.to(device)

net.train()

Net(
  (dp_one): Dropout(p=0.2, inplace=False)
  (dp_two): Dropout(p=0.2, inplace=False)
  (bn_one): BatchNorm2d(3, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv_one): Conv2d(3, 30, kernel_size=(3, 3), stride=(1, 1))
  (bn_two): BatchNorm2d(30, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv_two): Conv2d(30, 60, kernel_size=(3, 3), stride=(1, 1))
  (bn_three): BatchNorm2d(60, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv_three): Conv2d(60, 120, kernel_size=(3, 3), stride=(1, 1))
  (bn_four): BatchNorm2d(120, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc1): Linear(in_features=480, out_features=200, bias=True)
  (fc2): Linear(in_features=200, out_features=60, bias=True)
  (out): Linear(in_features=60, out_features=10, bias=True)
)

In [227]:
optimizer = torch.optim.Adam(net.parameters(), lr = 0.01)
criterion = nn.CrossEntropyLoss()

In [228]:
num_epochs = 5
net.train()

for epoch in range(num_epochs):  
    running_loss, running_items, running_right = 0.0, 0.0, 0.0
    for i, data in enumerate(train_loader, 0):
        inputs, labels = data[0].to(device), data[1].to(device)
        
        # обнуляем градиент
        optimizer.zero_grad()

        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # выводим статистику о процессе обучения
        running_loss += loss.item()
        running_items += len(labels)
        running_right += (labels == torch.max(outputs, 1)[1]).sum()
        
        # выводим статистику о процессе обучения
        if i % 300 == 0:    # печатаем каждые 300 mini-batches
            net.eval()
            
            print(f'Epoch [{epoch + 1}/{num_epochs}]. ' \
                  f'Step [{i + 1}/{len(train_loader)}]. ' \
                  f'Loss: {running_loss / running_items:.3f}. ' \
                  f'Acc: {running_right / running_items:.3f}', end='. ')
            running_loss, running_items, running_right = 0.0, 0.0, 0.0

            test_running_right, test_running_total = 0.0, 0.0
            for i, data in enumerate(valid_loader):
            
                test_outputs = net(data[0].to(device))
                test_running_total += len(data[1])
                test_running_right += (data[1].to(device) == torch.max(test_outputs, 1)[1]).sum()
            
            print(f'Test acc: {test_running_right / test_running_total:.3f}')
        
        net.train()
        
print('Training is finished!')

Epoch [1/5]. Step [1/1584]. Loss: 0.078. Acc: 0.067. Test acc: 0.120
Epoch [1/5]. Step [301/1584]. Loss: 0.066. Acc: 0.232. Test acc: 0.270
Epoch [1/5]. Step [601/1584]. Loss: 0.062. Acc: 0.282. Test acc: 0.330
Epoch [1/5]. Step [901/1584]. Loss: 0.059. Acc: 0.326. Test acc: 0.370
Epoch [1/5]. Step [1201/1584]. Loss: 0.057. Acc: 0.354. Test acc: 0.431
Epoch [1/5]. Step [1501/1584]. Loss: 0.055. Acc: 0.389. Test acc: 0.416
Epoch [2/5]. Step [1/1584]. Loss: 0.055. Acc: 0.333. Test acc: 0.474
Epoch [2/5]. Step [301/1584]. Loss: 0.053. Acc: 0.408. Test acc: 0.472
Epoch [2/5]. Step [601/1584]. Loss: 0.052. Acc: 0.422. Test acc: 0.491
Epoch [2/5]. Step [901/1584]. Loss: 0.052. Acc: 0.431. Test acc: 0.454
Epoch [2/5]. Step [1201/1584]. Loss: 0.051. Acc: 0.447. Test acc: 0.556
Epoch [2/5]. Step [1501/1584]. Loss: 0.050. Acc: 0.460. Test acc: 0.486
Epoch [3/5]. Step [1/1584]. Loss: 0.050. Acc: 0.467. Test acc: 0.498
Epoch [3/5]. Step [301/1584]. Loss: 0.049. Acc: 0.474. Test acc: 0.546
Epoch [3

In [234]:
resnet50 = models.resnet50(pretrained=True)
resnet50 = resnet50.to(device)

In [235]:
for param in list(resnet50.parameters())[:]:
    param.requires_grad = False

In [236]:
from torchsummary import summary

resnet50.fc = nn.Linear(2048, 12)

summary(resnet50.to(device), input_size=(3, 32, 32))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 64, 16, 16]           9,408
       BatchNorm2d-2           [-1, 64, 16, 16]             128
              ReLU-3           [-1, 64, 16, 16]               0
         MaxPool2d-4             [-1, 64, 8, 8]               0
            Conv2d-5             [-1, 64, 8, 8]           4,096
       BatchNorm2d-6             [-1, 64, 8, 8]             128
              ReLU-7             [-1, 64, 8, 8]               0
            Conv2d-8             [-1, 64, 8, 8]          36,864
       BatchNorm2d-9             [-1, 64, 8, 8]             128
             ReLU-10             [-1, 64, 8, 8]               0
           Conv2d-11            [-1, 256, 8, 8]          16,384
      BatchNorm2d-12            [-1, 256, 8, 8]             512
           Conv2d-13            [-1, 256, 8, 8]          16,384
      BatchNorm2d-14            [-1, 25

In [181]:
num_epochs = 5
resnet50.train()

for epoch in range(num_epochs):  
    running_loss, running_items, running_right = 0.0, 0.0, 0.0
    for i, data in enumerate(train_loader, 0):
        inputs, labels = data[0].to(device), data[1].to(device)
        
        # обнуляем градиент
        optimizer.zero_grad()

        outputs = resnet50(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # выводим статистику о процессе обучения
        running_loss += loss.item()
        running_items += len(labels)
        running_right += (labels == torch.max(outputs, 1)[1]).sum()
        
        # выводим статистику о процессе обучения
        if i % 300 == 0:    # печатаем каждые 300 mini-batches
            resnet50.eval()
            
            print(f'Epoch [{epoch + 1}/{num_epochs}]. ' \
                  f'Step [{i + 1}/{len(train_loader)}]. ' \
                  f'Loss: {running_loss / running_items:.3f}. ' \
                  f'Acc: {running_right / running_items:.3f}', end='. ')
            running_loss, running_items, running_right = 0.0, 0.0, 0.0

            test_running_right, test_running_total = 0.0, 0.0
            for i, data in enumerate(valid_loader):
            
                test_outputs = resnet50(data[0].to(device))
                test_running_total += len(data[1])
                test_running_right += (data[1].to(device) == torch.max(test_outputs, 1)[1]).sum()
            
            print(f'Test acc: {test_running_right / test_running_total:.3f}')
        
        resnet50.train()
        
print('Training is finished!')

Epoch [1/5]. Step [1/1584]. Loss: 0.083. Acc: 0.167. Test acc: 0.096
Epoch [1/5]. Step [301/1584]. Loss: 0.085. Acc: 0.096. Test acc: 0.090
Epoch [1/5]. Step [601/1584]. Loss: 0.085. Acc: 0.092. Test acc: 0.096
Epoch [1/5]. Step [901/1584]. Loss: 0.084. Acc: 0.093. Test acc: 0.096
Epoch [1/5]. Step [1201/1584]. Loss: 0.085. Acc: 0.093. Test acc: 0.097
Epoch [1/5]. Step [1501/1584]. Loss: 0.085. Acc: 0.095. Test acc: 0.091
Epoch [2/5]. Step [1/1584]. Loss: 0.083. Acc: 0.133. Test acc: 0.098
Epoch [2/5]. Step [301/1584]. Loss: 0.085. Acc: 0.088. Test acc: 0.095
Epoch [2/5]. Step [601/1584]. Loss: 0.084. Acc: 0.094. Test acc: 0.093
Epoch [2/5]. Step [901/1584]. Loss: 0.085. Acc: 0.092. Test acc: 0.092
Epoch [2/5]. Step [1201/1584]. Loss: 0.084. Acc: 0.093. Test acc: 0.097
Epoch [2/5]. Step [1501/1584]. Loss: 0.084. Acc: 0.095. Test acc: 0.093
Epoch [3/5]. Step [1/1584]. Loss: 0.084. Acc: 0.100. Test acc: 0.094
Epoch [3/5]. Step [301/1584]. Loss: 0.085. Acc: 0.088. Test acc: 0.089
Epoch [3

In [237]:
params_to_update = []
for name, param in resnet50.named_parameters():
    if param.requires_grad == True:
        params_to_update.append(param)


optimizer = torch.optim.Adam(params_to_update, lr=0.001)
criterion = nn.CrossEntropyLoss()

In [ ]:
num_epochs = 5
resnet50.train()

for epoch in range(num_epochs):  
    running_loss, running_items, running_right = 0.0, 0.0, 0.0
    for i, data in enumerate(train_loader, 0):
        inputs, labels = data[0].to(device), data[1].to(device)
        
        # обнуляем градиент
        optimizer.zero_grad()

        outputs = resnet50(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # выводим статистику о процессе обучения
        running_loss += loss.item()
        running_items += len(labels)
        running_right += (labels == torch.max(outputs, 1)[1]).sum()
        
        # выводим статистику о процессе обучения
        if i % 300 == 0:    # печатаем каждые 300 mini-batches
            resnet50.eval()
            
            print(f'Epoch [{epoch + 1}/{num_epochs}]. ' \
                  f'Step [{i + 1}/{len(train_loader)}]. ' \
                  f'Loss: {running_loss / running_items:.3f}. ' \
                  f'Acc: {running_right / running_items:.3f}', end='. ')
            running_loss, running_items, running_right = 0.0, 0.0, 0.0

            test_running_right, test_running_total = 0.0, 0.0
            for i, data in enumerate(valid_loader):
            
                test_outputs = resnet50(data[0].to(device))
                test_running_total += len(data[1])
                test_running_right += (data[1].to(device) == torch.max(test_outputs, 1)[1]).sum()
            
            print(f'Test acc: {test_running_right / test_running_total:.3f}')
        
        resnet50.train()
        
print('Training is finished!')

Epoch [1/5]. Step [1/1584]. Loss: 0.083. Acc: 0.033. Test acc: 0.092
Epoch [1/5]. Step [301/1584]. Loss: 0.066. Acc: 0.313. Test acc: 0.328
Epoch [1/5]. Step [601/1584]. Loss: 0.061. Acc: 0.369. Test acc: 0.366
Epoch [1/5]. Step [901/1584]. Loss: 0.061. Acc: 0.384. Test acc: 0.376
Epoch [1/5]. Step [1201/1584]. Loss: 0.060. Acc: 0.385. Test acc: 0.368
Epoch [1/5]. Step [1501/1584]. Loss: 0.059. Acc: 0.389. Test acc: 0.375
Epoch [2/5]. Step [1/1584]. Loss: 0.061. Acc: 0.267. Test acc: 0.394
Epoch [2/5]. Step [301/1584]. Loss: 0.058. Acc: 0.401. Test acc: 0.397
Epoch [2/5]. Step [601/1584]. Loss: 0.058. Acc: 0.408. Test acc: 0.407
Epoch [2/5]. Step [901/1584]. Loss: 0.058. Acc: 0.415. Test acc: 0.393
Epoch [2/5]. Step [1201/1584]. Loss: 0.058. Acc: 0.404. Test acc: 0.388
Epoch [2/5]. Step [1501/1584]. Loss: 0.058. Acc: 0.412. Test acc: 0.382
Epoch [3/5]. Step [1/1584]. Loss: 0.066. Acc: 0.267. Test acc: 0.398
Epoch [3/5]. Step [301/1584]. Loss: 0.057. Acc: 0.406. Test acc: 0.394
Epoch [3